In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[4]:

In [ ]:
get_ipython().magic('pylab inline')

###Demonstrate using the simulator for a surface simulation, deterministic <br>
integration.<br>
<br>
``Run time``: approximately 35 s (geodist step of local Connect) + ~5 min (workstation circa 2010).<br>
<br>
``Memory requirement``: < 1 GB

In[5]:

In [ ]:
from tvb.datatypes.cortex import Cortex
from tvb.datatypes.local_connectivity import LocalConnectivity

In [ ]:
from tvb.simulator.lab import *
LOG = get_logger('demo')

####Perform the simulation  

In[6]:

In [ ]:
LOG.info("Configuring...")

In[7]:

nitialise a Model, Coupling, and Connectivity.

In [ ]:
oscillator = models.Generic2dOscillator()

In [ ]:
white_matter = connectivity.Connectivity(load_default=True)
white_matter.speed = numpy.array([4.0])
white_matter_coupling = coupling.Linear(a=2 ** -9)

In[8]:

nitialise an Integrator

In [ ]:
heunint = integrators.HeunDeterministic(dt=2 ** -4)

In[9]:

nitialise some Monitors with period in physical time

In [ ]:
mon_tavg = monitors.TemporalAverage(period=2 ** -1)
mon_savg = monitors.SpatialAverage(period=2 ** -2)
#load the default region mapping
rm = region_mapping.RegionMapping(load_default=True)
#Initialise some Monitors with period in physical time
mon_eeg = monitors.EEG(load_default=True,
                        region_mapping=rm,
                        period=2**-2) 
#Bundle them
what_to_watch = (mon_tavg, mon_savg, mon_eeg)

In[10]:

nitialise a surface:<br>
irst define the function describing the "local" connectivity.

In [ ]:
grey_matter = LocalConnectivity(cutoff=40.0)
grey_matter.equation.parameters['sigma'] = 10.0
grey_matter.equation.parameters['amp'] = 1.0

hen a scaling factor, to adjust the strength of the local connectivity 

In [ ]:
local_coupling_strength = numpy.array([-0.0115])

inally, create a default cortex that includes the custom local connectivity.

In [ ]:
default_cortex = Cortex(load_default=True)
default_cortex.local_connectivity = grey_matter
default_cortex.coupling_strength = local_coupling_strength

In[11]:

nitialise Simulator -- Model, Connectivity, Integrator, Monitors, and surface.

In [ ]:
sim = simulator.Simulator(model=oscillator, connectivity=white_matter,
                          integrator=heunint, monitors=what_to_watch,
                          surface=default_cortex)
sim.configure()

In[12]:

In [ ]:
LOG.info("Starting simulation...")
#Perform the simulation
tavg_data = []
tavg_time = []
savg_data = []
savg_time = []
eeg_data = []
eeg_time = []

In [ ]:
for tavg, savg, eeg in sim(simulation_length=2 ** 6):
    if not tavg is None:
        tavg_time.append(tavg[0])
        tavg_data.append(tavg[1])
    
    if not savg is None:
        savg_time.append(savg[0])
        savg_data.append(savg[1])
    
    if not eeg is None:
        eeg_time.append(eeg[0])
        eeg_data.append(eeg[1])

In [ ]:
LOG.info("finished simulation")

####Plot pretty pictures of what we just did

In[13]:

ake the lists numpy.arrays for easier use.

In [ ]:
TAVG = numpy.array(tavg_data)
SAVG = numpy.array(savg_data)
EEG = numpy.array(eeg_data)

lot region averaged time series

In [ ]:
figure(1)
plot(savg_time, SAVG[:, 0, :, 0])
title("Region average")

lot EEG time series

In [ ]:
figure(2)
plot(eeg_time, EEG[:, 0, :, 0])
title("EEG")

urface movie, requires mayavi.malb

In [ ]:
if IMPORTED_MAYAVI:
    st = surface_timeseries(sim.surface, TAVG[:, 0, :, 0])
    
plot_local_connectivity(default_cortex)

how them

In [ ]:
show()